In [6]:
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
import anywidget
import traitlets
import jupyter
#from tweet_browser_test import tweet_browser as tb
import tweet_browser as tb
import voila
from matplotlib import pyplot as plt
from IPython.display import display, Javascript
import pandas as pd
import io
import math
import time

TWEETS_PER_PAGE = 50
JUPYTER_FILE_PATH = "../tree/images/"
# JUPYTER_FILE_PATH = "images/"

out = widgets.Output()

def startSession(file):
    if file['type'] == 'xls':
        data = pd.read_excel(io.BytesIO(file.content))
    else:
        data = pd.read_csv(io.BytesIO(file.content))
    s = tb.Session(data, False)

def autoStartSession(fileName):
    data = tb.parse_data(fileName)
    s = tb.Session(data, False)
    browser = Browser(s, out)

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[s.headerDict[j]])
    return result

class SearchBar(anywidget.AnyWidget):
    _esm = "anywidget/searchBar.js"
    _css = "anywidget/searchBar.css"
    value = traitlets.List([]).tag(sync=True)
    header = traitlets.Unicode("").tag(sync=True)
    header2 = traitlets.Unicode("").tag(sync=True)
    placeholder = traitlets.Unicode("").tag(sync=True)
    count = traitlets.Int(0).tag(sync=True)
    
class TweetDisplay(anywidget.AnyWidget):
    _esm = "anywidget/tweetDisplay.js"
    _css = "anywidget/tweetDisplay.css"
    value = traitlets.List([]).tag(sync=True)
    height = traitlets.Unicode("40vh").tag(sync=True)
    filePath = traitlets.Unicode(JUPYTER_FILE_PATH).tag(sync=True)
    
class DatasetDisplay(anywidget.AnyWidget):
    _esm = "anywidget/datasetDisplayPart.js"
    _css = "anywidget/misc.css"
    size = traitlets.Int().tag(sync=True)
    fileName = traitlets.Unicode().tag(sync=True)
    
class PageSelect(anywidget.AnyWidget):
    _esm = "anywidget/pageSelect.js"
    _css = "anywidget/pageSelect.css"
    tweetsPerPage = traitlets.Int(TWEETS_PER_PAGE).tag(sync=True)
    value = traitlets.CInt(1).tag(sync=True)
    totalTweets = traitlets.Int().tag(sync=True)
    changeSignal = traitlets.Int(0).tag(sync=True)
    filePath = traitlets.Unicode(JUPYTER_FILE_PATH).tag(sync=True)

# File upload
fileUp = widgets.widgets.FileUpload(
    accept='.csv, .txt, .xls, .tsv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description='Change Dataset'
)

# fileUp = FileInput(accept=".csv,.txt,.xls,.tsv", label="Change Dataset", style_="background-color: black;", prepend_icon="")

fileUp.add_class("change-input")
datasetDisplayCustom = DatasetDisplay()
datasetDisplayCustom.add_class("text-block")
fileUpBar = widgets.Box([datasetDisplayCustom, fileUp])
fileUpBar.add_class("dataset-display")

class SortBar(anywidget.AnyWidget):
    _esm = "anywidget/sortBar.js"
    _css = "anywidget/sortBar.css"
    sortScope = traitlets.Unicode("Displayed Examples").tag(sync=True)
    sortColumn = traitlets.Unicode("None").tag(sync=True)
    sortOrder = traitlets.Unicode("DESC").tag(sync=True)
    filePath = traitlets.Unicode(JUPYTER_FILE_PATH).tag(sync=True)

class ToggleSwitch(anywidget.AnyWidget):
    _esm = "anywidget/toggleSwitch.js"
    _css = "anywidget/toggleSwitch.css"
    value = traitlets.Int(0).tag(sync=True)
    label = traitlets.Unicode("").tag(sync=True)
    calendarStart = traitlets.Unicode("").tag(sync=True)
    calendarEnd = traitlets.Unicode("").tag(sync=True)

class ParameterDisplay(anywidget.AnyWidget):
    _esm = "anywidget/parameterDisplay.js"
    _css = "anywidget/parameterDisplay.css"
    headers = traitlets.List().tag(sync=True)
    value = traitlets.List().tag(sync=True)
    notFound = traitlets.Unicode().tag(sync=True)
    firstWord = traitlets.Unicode().tag(sync=True)
    secondWord = traitlets.Unicode().tag(sync=True)

class AiSummary(anywidget.AnyWidget):
    _esm = "anywidget/aiSummary.js"
    _css = "anywidget/aiSummary.css"
    value = traitlets.List().tag(sync=True)
    sentenceNums = traitlets.List().tag(sync=True)
    selected = traitlets.CInt(0).tag(sync=True)
    changeSignal = traitlets.Int(0).tag(sync=True)

class SearchHistory(anywidget.AnyWidget):
    _esm = "anywidget/searchHistory.js"
    _css = "anywidget/searchHistory.css"
    filePath = traitlets.Unicode(JUPYTER_FILE_PATH).tag(sync=True)
    current = traitlets.Int(0).tag(sync=True)
    count = traitlets.Int(0).tag(sync=True)
    changeSignal = traitlets.Int(0).tag(sync=True)
    
class StoredSearch():
    def __init__(self, geography=[], username=[], fromDate=None, toDate=None, allowRetweets=False, exclude=[], mustInclude=[], containOneOf=[]):
        self.geography = geography
        self.userName = username
        self.fromDate = fromDate
        self.toDate = toDate
        self.allowRetweets = allowRetweets
        self.exclude = exclude
        self.mustInclude = mustInclude
        self.containOneOf = containOneOf

class Browser:
    def __init__(self, s, out):
        self.s = s
        self.screen = "main"
        self.colHeaders = list(s.headerDict.keys())
        self.history = []
        self.createWidgets()
        self.search(None)
        self.resetDisplay()
        # self.history = [StoredSearch()]
        
    def resetDisplay(self, b = None):
        out.clear_output(True)
        with out:
            if self.screen == "main":
                display(self.mainPage)
                self.getTweets(None)
            elif self.screen == "advanced":
                display(self.closeButton)
                display(self.advancedPage)
                # display(self.script)
                # display(self.searchButton)
            elif self.screen == "summary":
                display(self.closeButton)
                display(self.aiSummary)
            elif self.screen == "contributingTweets":
                self.getSummaryTweets(None)
                display(self.closeButton)
                display(self.contributingTweets)
            # display(self.debugText)

            
    def search(self, b):
        self.s.currentSet = self.s.base
        
        for i in range(self.geography.count):
            self.s.filterBy("State", self.geography.value[i].lower().capitalize())
        for i in range(self.userName.count):
            self.s.filterBy("SenderScreenName", self.userName.value[i])
        if(self.fromDate.value != None and self.toDate.value != None):
            self.s.filterDate(self.fromDate.value.strftime('%Y-%m-%d'), self.toDate.value.strftime('%Y-%m-%d'))
        if(self.allowRetweets.value == 0):
            self.s.removeRetweets()
        if(self.exclude.count > 0):
            self.s.exclude(self.exclude.value)
        if(self.mustInclude.count > 0):
            self.s.searchKeyword(self.mustInclude.value, False)
        if(self.containOneOf.count > 0):
            self.s.searchKeyword(self.containOneOf.value, True)
        self.getTweets(b)

        currSearch = StoredSearch(self.geography.value, self.userName.value, self.fromDate.value, self.toDate.value, self.allowRetweets.value, self.exclude.value, self.mustInclude.value, self.containOneOf.value)
        self.history.append(currSearch)
        
        self.searchHistory.current = self.searchHistory.count
        self.searchHistory.count += 1
        #change the text on the search button after first search
        if len(self.history) > 1:
            self.advancedButton.description = "Modify Search"

        self.updateSearchParams(b)
    
    def getTweets(self, b):
        if self.s.currentSet == self.s.base:
            return
        dataSet = self.s.getCurrentSubset()
        pageNum = self.pageSelect.value
        self.pageSelect.totalTweets = len(dataSet)
        
        self.pageSelect.changeSignal += 1

        tempArr = []
        sorted = self.getSortedTweets(pageNum)
        for i in range(len(sorted)):   
            tempArr.append(sorted.iloc[i].to_json())
            
        self.tweetDisplay.value = tempArr

    def getSummaryTweets(self, b):
        pageNum = self.pageSelectAi.value - 1 # convert to 0 indexing
        self.summaryLine.value = self.aiSummary.value[pageNum]
        ans = self.s.getCurrentSubset()
        tweets = self.aiSummary.sentenceNums[pageNum]
        ans = ans.reset_index(drop=True).iloc[tweets]
        tempArr = []
        for i in range(len(ans)):
            tempArr.append(ans.iloc[i].to_json())
        self.summaryDisplay.value = tempArr
        self.pageSelectAi.changeSignal += 1
    
    def getSortedTweets(self, pageNum):
        ans = self.s.getCurrentSubset()
            
        if self.sortBar.sortColumn == "None":
            return ans.iloc[(pageNum-1) * TWEETS_PER_PAGE : min(pageNum * TWEETS_PER_PAGE, len(ans))]

        asc = True
        na_pos = "first"
        column = self.sortBar.sortColumn
        keyFunc = None
            
        if (column == "Username" or column == "SenderScreenName"):
            keyFunc = userNameToLower
        if (self.sortBar.sortOrder == "DESC"):
            asc = False
            na_pos = "last"
        if (self.sortBar.sortScope == "Entire Dataset"):
            ans = ans.sort_values(by=[column], ascending=asc, na_position = na_pos, key=keyFunc)
            ans = ans.iloc[(pageNum-1) * TWEETS_PER_PAGE : min(pageNum * TWEETS_PER_PAGE, len(ans))]
        elif (self.sortBar.sortScope == "Displayed Examples"):
            sortedPage = ans.iloc[(pageNum-1) * TWEETS_PER_PAGE : min(pageNum * TWEETS_PER_PAGE, len(ans))]
            sortedPage = sortedPage.sort_values(by=[column], ascending=asc, na_position = na_pos, key=keyFunc)
            ans = sortedPage
        else:
            with out:
                print("An Error has occured while sorting")
                self.debugText.value = "Sorting error"
        return ans
        
    def createWidgets(self):
        self.advancedButton = widgets.Button(description='Search & Filter', icon = "search")
        self.advancedButton.add_class("generic-button").add_class("modify-search")
        self.advancedButton.on_click(self.openSearchMenu)
        self.tweetDisplay = TweetDisplay(height="60vh")
        self.datasetDisplay = fileUpBar
        self.pageSelect = PageSelect()
        self.pageSelect.observe(self.getTweets, names=["value"])
        self.generateSummary = widgets.Button(description="Generate AI Summary")
        self.generateSummary.add_class("generic-button")
        self.generateSummary.on_click(self.generateAiSummary)
        self.sortBar = SortBar()
        self.sortBar.observe(self.getTweets, names=["sortScope", "sortColumn", "sortOrder"])
        self.optionsBar = widgets.Box(children = [self.datasetDisplay, self.pageSelect, self.generateSummary, self.sortBar])
        self.optionsBar.layout = widgets.Layout(align_items = "center", justify_content = "space-between", width = "100%")
        self.searchedKeywords = ParameterDisplay(firstWord = "Searched", secondWord = "Keywords", headers = ["Must Include", "Contain one of", "Exclude"], notFound = 'To enter keywords, click "Search & Filter"')
        self.appliedFilters = ParameterDisplay(firstWord = "Applied", secondWord = "Filters", headers = ["Date Range", "Geography", "Username", "Allow Retweets"], notFound = 'To enter filters, click "Search & Filter"')
        self.paramDisplay = widgets.HBox([self.searchedKeywords, self.appliedFilters, self.advancedButton])
        self.randomSelection = widgets.VBox([self.optionsBar, self.tweetDisplay])
        self.tabs = widgets.Tab(children=[self.randomSelection], titles=["Random Selection"])
        self.mainPage = widgets.VBox([self.paramDisplay, self.tabs])

        self.debugText = widgets.HTML("test")

        self.makeAdvancedPage()
        self.makeAiSummaryPage()
        self.makeSummaryContributionPage()

    def makeAdvancedPage(self):
        self.searchButton = widgets.Button(description='Search', icon="search")
        self.hiddenButton = widgets.Button()
        self.hiddenButton.add_class("hidden-button") # work around for syncing search when the user still has input in the search bars
        self.hiddenButton.on_click(self.search)
        self.searchButton.add_class("generic-button").add_class("search-button")
        self.clearButton = widgets.Button(description='Clear All')
        self.clearButton.on_click(self.clearSettings)
        self.clearButton.add_class("clear-button")
        self.searchHistory = SearchHistory()
        self.searchHistory.observe(self.loadSearch, names=["changeSignal"])
        self.bottomBar = widgets.HBox([self.searchHistory, self.clearButton, self.searchButton, self.hiddenButton], layout = widgets.Layout(justify_content = "flex-end"))
        self.keyWordSearch = widgets.HTML(value = "<b>Keyword Search<b/>")
        self.keyWordSearch.add_class("keyword-search")
        self.mustInclude = SearchBar(header = "Must include all", header2="(AND)", placeholder='e.g. “civil null” means each post in the result must contain the word “civil” and “null”')
        self.containOneOf = SearchBar(header = "Must include one Of", header2="(OR)", placeholder='e.g. “census penny” means each post in the result must contain either “census” or “penny” or both')
        self.exclude = SearchBar(header = "Must not include", header2="(NOT)", placeholder='e.g. “toxic ban” means none of the posts in the result contains the word “toxic” and “ban”')
        self.searches = widgets.VBox([self.keyWordSearch, self.mustInclude, self.containOneOf, self.exclude])
        self.searches.add_class("search-box")
        self.closeButton = widgets.Button(description = 'X')
        self.closeButton.add_class("close-button")
        self.closeButton.on_click(self.closeSearchMenu)
        self.filterBy = widgets.HTML(value = "<b>Filter By<b/>")
        self.dateRange = widgets.HTML(value = "<b style='font-size: 1.17em;'>Date Range <b/>")
        self.fromDate = widgets.DatePicker(description = "From")
        self.toDate = widgets.DatePicker(description = "To")
        minDate = self.s.findMinDate().strftime("%Y-%m-%d")
        maxDate = self.s.findMaxDate().strftime("%Y-%m-%d")
        self.fromDate.add_class("date-constraint") # The script to set the elements attribute is attached to the toggleSwitch widget
        self.toDate.add_class("date-constraint") # This was done for convenience and should be changed later

        self.dateBox = widgets.VBox([self.dateRange, self.fromDate, self.toDate])
        self.allowRetweets = ToggleSwitch(label = "Include reposts", calendarStart = minDate, calendarEnd = maxDate) # TODO: move calendar script somwhere else
        self.geography = SearchBar(header = "Geography", placeholder = "Search")
        self.userName = SearchBar(header = "Username", placeholder = "Search")
        self.filterBox = widgets.VBox([self.filterBy, self.dateBox, self.allowRetweets, self.geography, self.userName])
        self.filterBox.add_class("filter-box")
        self.advancedBox = widgets.HBox([self.searches, self.filterBox])
        self.advancedBox.add_class("advanced-box")
        self.advancedPage = widgets.VBox([self.advancedBox, self.bottomBar])
    
    def makeAiSummaryPage(self):
        self.aiSummary = AiSummary(sentenceNums=[[1, 3, 4, 8, 11, 13, 21, 24, 27, 31, 35, 36, 37, 38, 39], [6, 12, 16, 22], [3, 7, 28], [14, 25, 27, 43]], value=["The tweets largely focus on the importance of participating in the U.S. Census, emphasizing that it is crucial to ensure everyone is counted in order to determine resources and representation.", "Many tweets emphasize that the census should include everyone, regardless of their race, ethnicity, citizenship status, or any other characteristic.", "Some tweets highlight the role of the census in tracking population growth and demographic changes.", "There are also discussions about the inclusion or exclusion of non-citizens or undocumented immigrants from the census"])
        self.aiSummary.observe(self.showContributingTweets, names=["changeSignal"])
    
    def makeSummaryContributionPage(self):
        self.backArrow = widgets.Button(icon="solid fa-arrow-left").add_class("back-button")
        self.backArrow.on_click(self.generateAiSummary)
        self.summaryLine = widgets.HTML(value="placeholder").add_class("selected-sentence")
        self.pageSelectAi = PageSelect()
        self.pageSelectAi.observe(self.getSummaryTweets, names=["value"])
        self.summaryDisplay = TweetDisplay(height="70vh")
        leftBar = widgets.VBox([self.backArrow, self.summaryLine, self.pageSelectAi]).add_class("left-bar")
        summaryBar = widgets.HBox([leftBar, self.summaryDisplay])
        self.contributingTweets = widgets.VBox([widgets.HTML(value="<h1>Contributing Tweets</h1>"), widgets.HTML(value="Here are the tweets that contribute to this part of the summary"), summaryBar])
    
    def openSearchMenu(self, change):
        self.screen = "advanced"
        self.resetDisplay()
        
    def generateAiSummary(self, change):
        self.screen = "summary"
        self.resetDisplay()

    def clearSettings(self, change):
        self.restoreSearch(self.history[0])

    def loadSearch(self, change):
        index = self.searchHistory.current
        self.restoreSearch(self.history[index])
        self.resetDisplay()
        
    def restoreSearch(self, settings):
        self.geography.value = settings.geography
        self.geography.count = len(settings.geography)
        self.userName.value = settings.userName
        self.userName.count = len(settings.userName)
        self.fromDate.value = settings.fromDate
        self.toDate.value = settings.toDate
        self.allowRetweets.value = settings.allowRetweets
        self.exclude.value = settings.exclude
        self.exclude.count = len(settings.exclude)
        self.mustInclude.value = settings.mustInclude
        self.mustInclude.count = len(settings.mustInclude)
        self.containOneOf.value = settings.containOneOf
        self.containOneOf.count = len(settings.containOneOf)
        self.resetDisplay()

    def showContributingTweets(self, change):
        self.screen = "contributingTweets"
        self.pageSelectAi.totalTweets = self.pageSelectAi.tweetsPerPage * (len(self.aiSummary.sentenceNums)-1) + 1
        self.pageSelectAi.value = self.aiSummary.selected + 1 # convert from 0 indexing
        self.resetDisplay()
    
    def updateSearchParams(self, change):
        val1 = val2 = val3 = ""
        if(self.mustInclude.count > 0):
            val1 = ', '.join(self.mustInclude.value)
        if(self.containOneOf.count > 0):
            val2 = ', '.join(self.containOneOf.value)
        if(self.exclude.count > 0):
            val3 = ', '.join(self.exclude.value)
        self.searchedKeywords.value = [val1, val2, val3]
        selectedDates = ''
        if(self.fromDate.value != None and self.toDate.value != None):
            selectedDates = str(self.fromDate.value) + " to " + str(self.toDate.value)
        geo = usrname = ""
        if (self.geography.count > 0):
            geo = ', '.join(self.geography.value)
        if (self.userName.count > 0):
            usrname = ', '.join(self.userName.value)
        retweets = "yes" if self.allowRetweets.value > 0 else "no"
        self.appliedFilters.value = [selectedDates, geo, usrname, retweets]
        self.closeSearchMenu(change)

    def closeSearchMenu(self, change):
        self.screen = "main"
        self.resetDisplay()
    
def fileHandler(change):
    startSession(fileUp.value[0])
    
def userNameToLower(input):
    return input.str.lower()

# with out:
#     display(fileUpBar)

autoStartSession("allCensus_sample.csv")

fileUp.observe(fileHandler, names=["value"])

out
#TODO: make serach and filter button change to modify search

Output()